# Script d'import des données de bio.tools dans Neo4j

## Initialisation de la connexion avec la base de donnée:

In [1]:
from py2neo import *
import json
IP="bolt://localhost:7687"
USER="neo4j"
PASSWORD="pass"
graph_db = Graph(IP, auth=(USER, PASSWORD))

Nettoyage de la BDD par morceaux pour éviter le chargement de toutes les données en RAM:

In [2]:
graph_db.run("""CALL apoc.periodic.commit("MATCH (n) DETACH DELETE n RETURN Count(*) LIMIT 1000", {limit:1000});""")

updates,executions,runtime,batches,failedBatches,batchErrors,failedCommits,commitErrors,wasTerminated
153690,1,12,2,0,{},0,{},false


### Commande naïve décortiquant le JSON en utilisant UNWIND

apoc.periodic.iterate permet de découper la liste des outils présente dans le fichier JSON. IL prend une première requête en arguments qui doit renvoyer une liste d'élément. La deuxième requête s'execute sur chaque élément de la liste.
batchSize:1000 permet de faire un commit tout les 1000 éléments de la liste.

UNWIND parcourt les éléments d'une liste, il a l'inconvénient de mal supporter les imbrications de boucles (nécéssite un renomage des éléments (WITH) à chaque imbriation (et difficile de mettre des boucles à la suite)
    
```
   commande = """CALL apoc.periodic.iterate(
    "CALL apoc.load.json('file:///datatest.json') YIELD value",
    "UNWIND value.list AS elt
        MERGE (t:Tool {name: elt.name})
        ON CREATE SET t.description= elt.description, 
                    t.homepage= elt.homepage, 
                    t.toolLanguage= elt.language, 
                    t.toolOS= elt.operatingSystem, 
                    t.toolType= elt.toolType
        
        WITH t AS theToolNode, elt AS theElt
        UNWIND theElt.function AS fun
            MERGE (f:Function {uid: apoc.create.uuid()})
            MERGE (theToolNode)-[:hasFunction]->(f)
            WITH f AS theFunctionNode, fun AS theFunction, theToolNode AS theToolNode
                UNWIND theFunction.input AS input
                UNWIND theFunction.output AS output
                
                    MERGE (i:IO {name: input.data['term']})
                    ON CREATE SET i.name= input.data['term']
                    MERGE (theFunctionNode)<-[r:inputOf]-(i)
                    
                    MERGE (o:IO {name: output.data['term']})
                    ON CREATE SET o.name= output.data['term']
                    MERGE (theFunctionNode)-[:outputOf]->(o)",
                    
    {batchSize:10, iterateList:true});
"""
res = graph_db.run(commande)
```

### Commande d'importation en parcourant le JSON et à partir d'une BDD vide

***Le graphe doit être vide avant d'executer la requête car les CREATE présents dupliqueraient certains noeuds/relations***

Cette requête utilise des boucles FOREACH plutôt que des UNWIND, qui sont bien plus facile à manipuler
La première requête renvoie la liste d'outils présents dans le JSON, et pour chaque outils on lit ses données et on les importe dans le graphe.

Remarques:
- Un noeud/une relation ne peut stocker que des types élémentaires (String, Int) et des listes de ceux-ci
- J'ai fait le choix de créer des noeuds annexes autour des outils pour garder la structure présente dans la base de donnée de bio.tools (par exemple un noeud 'link' stockant les infos sur un lien exeterne).
- Je n'ai pas importé des données en lien avec les 'community' car je n'ai pas encore trouvé d'exemple pour voir la structure du JSON à cet endroit
- Un batchsize de 10000 importe les données en 3 min sur mon pc. (environ 170.000 noeuds et 360.000 relations)

In [6]:
# Nettoyage avant importation
graph_db.run("""CALL apoc.periodic.commit("MATCH (n) DETACH DELETE n RETURN Count(*) LIMIT 10000", {limit:10000});""")

#voir page 17 de l'api pour exemple sympa
commande = """CALL apoc.periodic.iterate(
    "CALL apoc.load.json('file:///datatest.json', '$.list') YIELD value RETURN value AS tool",
    "MERGE (t:Tool {name: tool.name})
        ON CREATE SET t.description= tool.description, 
                    t.homepage= tool.homepage,
                    t.license= tool.license,
                    t.collectionID= tool.collectionID,
                    t.maturity= tool.maturity,
                    t.cost= tool.cost,
                    t.accessibility= tool.accessibility,
                    // Check if complex datatype for elixir..
                    t.elixirPlatform= tool.elixirPlatform,
                    t.elixirNode= tool.elixirNode,
                    t.elixirCommunity= tool.elixirCommunity,
                    //TODO add community (Map mais pas dans l'exemple page 17)
                    //t.community = tool.community,
                    t.owner = tool.owner,
                    t.additionDate= tool.additionDate,
                    t.lastUpdate= tool.lastUpdate,
                    t.validated= tool.validated,
                    t.homepageStatus= tool.homepage_status,
                    t.elixirBadge= tool.elixir_badge,
                    t.confidenceFlag= tool.confidence_flag
                    
        
        foreach (function in tool.function |
            CREATE (f:Function {note: function.note, cmd: function.cmd})
            MERGE (t)-[:hasFunction]-(f)
            
            //add the operations of the functions
            foreach(operation in function.operation |
                MERGE (op:Operation {term: operation.term, uri: operation.uri})
                MERGE (f)-[:doOperation]-(op)
            )
            
            //add the inputs of the function
            foreach(input in function.input |
                MERGE (i:IO {term: input.data.term, uri: input.data.uri})
                MERGE (i)-[rel:inputOf]->(f)
                SET rel.format = []
                // the formats are linked to the function via the relation
                foreach(format in input.format |
                    SET rel.format = rel.format + apoc.convert.toJson({term:format.term, uri: format.uri})
                )
            )

            //add the outputs of the function
            foreach(output in function.output |
                MERGE (o:IO {term: output.data.term, uri: output.data.uri})
                MERGE (o)<-[rel:outputOf]-(f)
                SET rel.format = []
                // the formats are linked to the function via the relation
                foreach(format in output.format |
                    SET rel.format = rel.format + apoc.convert.toJson({term:format.term, uri: format.uri})
                )
            )
        )
            
        //add tooltype of the node
        SET t.toolType=[]
        foreach (toolType in tool.toolType |
            SET t.toolType = t.toolType + toolType
        )

        //add the topics
        foreach(topic in tool.topic |
            MERGE (top:Topic {term: topic.term, uri:topic.uri})
            MERGE (t)-[:Topic]-(top)
        )

        //add OSs
        foreach(operatingSystem in tool.operatingSystem |
            MERGE (os:OS {name: operatingSystem})
            MERGE (t)-[:OS]-(os)
        )

        //add languages
        foreach(language in tool.language |
            MERGE (l:Language {name:language})
            MERGE (t)-[:Language]-(l)
        )

        //add links
        foreach (link in tool.link |
            CREATE (l:Link {url: link.url, type: link.type, note: link.note})
            CREATE (t)-[:Link]->(l)
        )

        //add publications (ids of the publication in the relation, then metadata in the node
        foreach(pub in tool.publication |
            CREATE (p:Publication {title: pub.metadata.title,
                                abstract: pub.metadata.abstract,
                                date: pub.metadata.date,
                                citationCount: pub.metadata.citationCount,
                                authors: [], // authors is a list of Map{name->String}
                                journal: pub.metadata.journal })
            foreach(author in pub.metadata.authors | SET p.authors= p.authors + author['name']) // create the list of authors

            CREATE (t)-[:Publication {doi: pub.doi,
                                    pmid: pub.pmid,
                                    pmcid: pub.pmcid,
                                    type: pub.type,
                                    version: pub.version,
                                    note: pub.note} ]   ->(p)
        )

        //credits
        foreach(credit in tool.credit |
            CREATE (cred:Credit {name: credit.name,
                                email: credit.email,
                                url: credit.url,
                                orcidid: credit.orcidid,
                                gridid: credit.gridid,
                                rorid: credit.rorid,
                                fundrefid: credit.fundredid,
                                typeEntity: credit.typeEntity,
                                typeRole: credit.typeRole,
                                note: credit.note })
            CREATE (t)-[:Credit]->(cred)
        )

        //edit permission
        CREATE (ep:EditPermission {type: tool.editPermission.type,
                                   authors: tool.editPermission.authors })
        CREATE (t)-[:EditPermission]->(ep)

        //downloads
        foreach(dl in tool.download |
            CREATE (d:Download {url: dl.url, type: dl.type, note: dl.note, version: dl.version})
            CREATE (t)-[:Download]->(d)
        )

        //documentation
        foreach(dc in tool.documentation |
            CREATE (d:Documentation {url:dc.url, type: dc.type, note: dc.note})
            CREATE (t)-[:Documentation]->(d)
        )
        ",
        {batchSize:1000, iterateList:true}) YIELD timeTaken, batches, committedOperations RETURN timeTaken, batches, committedOperations
    """
res = graph_db.run(commande)
print(res.to_table());

 timeTaken | batches | committedOperations 
-----------|---------|---------------------
         1 |       1 |                 300 



### Quelques infos sur la base de donnée une fois importée:

In [15]:
print(graph_db.run("""MATCH (n) RETURN Labels(n) AS Node_type, Count(n) AS Count ORDER BY Node_type ASC""").to_table())

 Node_type          | Count 
--------------------|-------
 ['Credit']         | 30025 
 ['Documentation']  | 14783 
 ['Download']       |  5213 
 ['EditPermission'] | 22240 
 ['Function']       | 23104 
 ['IO']             |   559 
 ['Language']       |    44 
 ['Link']           | 11347 
 ['Metadata']       | 23178 
 ['OS']             |     3 
 ['Operation']      |   694 
 ['Publication']    | 23178 
 ['Tool']           | 22062 
 ['ToolType']       |    15 
 ['Topic']          |   423 



In [16]:
print(graph_db.run("""MATCH ()-[r]->() RETURN Type(r) AS Relation_type, Count(r) AS Count ORDER BY Relation_type ASC""").to_table())

 Relation_type  | Count 
----------------|-------
 Credit         | 30025 
 Documentation  | 14783 
 Download       |  5213 
 EditPermission | 22240 
 Language       | 18542 
 Link           | 11347 
 Metadata       | 23178 
 OS             | 35123 
 Publication    | 23178 
 ToolType       | 22450 
 Topic          | 68651 
 doOperation    | 48769 
 hasFunction    | 23104 
 inputOf        |  4294 
 outputOf       |  4092 

